## Step 2: Augment database with asset data view and supporting metadata

#### For all elements/assets with PIPoints, create a default Data View having as columns all those PIPoints

Data View column names are the attribute names in AF, so elements sharing a template have identical Data Views modulo missing streams

This notebook also creates specialized Data Views:

* Some are subsets of the default Data View, per asset
* Others are multi-assets version of the above  

#### After running this notebook, the graph has all the necessary information for Step 2: projection of tags/metadata on streams and Data View creation 

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import yaml
import pprint
from collections import OrderedDict

## Input Parameters

In [2]:
config_file = "config-cmu.yaml"
# config_file = "config-windfarm.yaml"  # 
# config_file = "config-acad-prod-desc-v2.yaml"

In [3]:
with open(config_file) as f:
    config = yaml.safe_load(f)
# config

In [4]:
# Input parameters

database_name = config["db"]["database_name"]
root_element = config["db"]["root_element"]
ocs_asset_db = dv_id_prefix = config["ocs"]["configuration"]["asset_db"]

# Each assets in a Hub dataset is endowed with default Data View (DV) "All_Columns" having all the asset streams
# The next line of code defines a dictionary to build all default (DV)
# If <column list> is empty list [], include all streams in Data View
dv_defs = {"Default": ("all_columns", [], lambda id: True)}
has_no_dv = lambda id: False


# YAML config file may embed additional code to define additional custom dataviews
try:
    exec(config["custom_dvs"])
except:
    pass

dv_defs

{'Default': ('all_columns', [], <function __main__.<lambda>(id)>)}

In [5]:
sample_transport = RequestsHTTPTransport(
    url=config["graphql"]["endpoint"], verify=False, retries=3
)
client = Client(transport=sample_transport, fetch_schema_from_transport=True)

In [6]:
db_query = gql(
    """
query DatabaseId($database: String) {
    Database(name: $database) {
        name
        id
        asset_db
    }
}
"""
)
print(f"database_name={database_name}")
db = client.execute(db_query, variable_values={"database": database_name})
print(json.dumps(db, indent=4))

database_name=CMU_UnitOps
{
    "Database": [
        {
            "name": "CMU_UnitOps",
            "id": "cmu.unitops:F1RDcFWpYyj-GkWKwoVLgdSfpwlJzOM9zJAEebSKeNcat3SgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRB",
            "asset_db": "cmu.unitops"
        }
    ]
}


In [7]:
db_id = db["Database"][0]["id"]
asset_db = db["Database"][0]["asset_db"]
asset_db, db_id

('cmu.unitops',
 'cmu.unitops:F1RDcFWpYyj-GkWKwoVLgdSfpwlJzOM9zJAEebSKeNcat3SgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRB')

In [11]:
root_query = gql(
    """
query RootElementTree($root : String, $asset_db: String) {
    Element(name: $root, asset_db: $asset_db) {
        name
        id
        has_element {
            name
            id
            has_dynamic {
                  name
                  stream_name
                  id
                  pointsource
            }
        }
    }
}
"""
)
j = client.execute(
    root_query, variable_values={"root": root_element, "asset_db": ocs_asset_db}
)
print(json.dumps(j, indent=4))

{
    "Element": [
        {
            "name": "cmu_unitops1",
            "id": "cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpw3eu3vXz56BGoMAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMQ",
            "has_element": [
                {
                    "name": "wk1_F20",
                    "id": "cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIw",
                    "has_dynamic": [
                        {
                            "name": "WP T Hot Out",
                            "stream_name": "cmu_unitops1.wk1_F20_data_values_container.WP T Hot Out",
                            "id": "cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw2vc21H241lEzdlPxaOdnxwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIE9VVA",
                            "pointsource": "wk1_F20_data_values_type"
                        },
      

In [12]:
elements = j["Element"][0]["has_element"] 
len(elements), elements

(7,
 [{'name': 'wk1_F20',
   'id': 'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIw',
   'has_dynamic': [{'name': 'WP T Hot Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot Out',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw2vc21H241lEzdlPxaOdnxwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIE9VVA',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Hot In',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot In',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw9f4NRI2t51QJ4H8YX5ZQCgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIElO',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Cold Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Cold Out',
     '

In [13]:
all_elements = elements
for element in elements:
    j = client.execute(root_query, variable_values={"root": element["name"]})
    all_elements += j["Element"][0]["has_element"] 
all_elements

[{'name': 'wk1_F20',
  'id': 'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIw',
  'has_dynamic': [{'name': 'WP T Hot Out',
    'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot Out',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw2vc21H241lEzdlPxaOdnxwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIE9VVA',
    'pointsource': 'wk1_F20_data_values_type'},
   {'name': 'WP T Hot In',
    'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot In',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw9f4NRI2t51QJ4H8YX5ZQCgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIElO',
    'pointsource': 'wk1_F20_data_values_type'},
   {'name': 'WP T Cold Out',
    'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Cold Out',
    'id': 'cmu.unitops

In [14]:
len(all_elements), all_elements

(7,
 [{'name': 'wk1_F20',
   'id': 'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIw',
   'has_dynamic': [{'name': 'WP T Hot Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot Out',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw2vc21H241lEzdlPxaOdnxwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIE9VVA',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Hot In',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot In',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw9f4NRI2t51QJ4H8YX5ZQCgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIElO',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Cold Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Cold Out',
     '

In [15]:
elements = all_elements
len(elements), elements

(7,
 [{'name': 'wk1_F20',
   'id': 'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIw',
   'has_dynamic': [{'name': 'WP T Hot Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot Out',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw2vc21H241lEzdlPxaOdnxwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIE9VVA',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Hot In',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Hot In',
     'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwhPycwJz36hGoOAANOjL3Qw9f4NRI2t51QJ4H8YX5ZQCgUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxXSzFfRjIwfFdQIFQgSE9UIElO',
     'pointsource': 'wk1_F20_data_values_type'},
    {'name': 'WP T Cold Out',
     'stream_name': 'cmu_unitops1.wk1_F20_data_values_container.WP T Cold Out',
     '

In [16]:
len(set([i["name"] for i in elements]))

7

In [17]:
d = {}
for e in elements:
    d[e["name"]] = e
d.keys()

dict_keys(['wk1_F20', 'Membrane', 'HX Cycle1 F20', 'F20_Wk1_PSA', 'F20_wk1_Membrane', 'F20_PSA_v2', 'CMU_controls_hxer_steptest'])

In [18]:
elements = [d[i] for i in sorted([i for i in d])]
elements

[{'name': 'CMU_controls_hxer_steptest',
  'id': 'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVA',
  'has_dynamic': [{'name': 'Hot Water Out',
    'stream_name': 'cmu_unitops1.CMU_controls_hxer_steptest_data_values_container.Hot Water Out',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwTPSBbXfLJFQJePjG_TcllAUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVHxIT1QgV0FURVIgT1VU',
    'pointsource': 'CMU_controls_hxer_steptest_data_values_type'},
   {'name': 'Hot Water In',
    'stream_name': 'cmu_unitops1.CMU_controls_hxer_steptest_data_values_container.Hot Water In',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwgPctt0f72V00TJF_JVAxrQUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVHxIT1QgV0FURVIgSU4',
    'pointsource': 

In [19]:
elements_info = [
    (
        elements[i]["name"],
        elements[i]["id"],
        [
            j for j in elements[i]["has_dynamic"] if j["pointsource"] != "AZURE"
        ],  # remove future tag
    )
    for i in range(len(elements))
    if len(elements[i]["has_dynamic"]) > 0
]
elements_info

[('CMU_controls_hxer_steptest',
  'cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVA',
  [{'name': 'Hot Water Out',
    'stream_name': 'cmu_unitops1.CMU_controls_hxer_steptest_data_values_container.Hot Water Out',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwTPSBbXfLJFQJePjG_TcllAUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVHxIT1QgV0FURVIgT1VU',
    'pointsource': 'CMU_controls_hxer_steptest_data_values_type'},
   {'name': 'Hot Water In',
    'stream_name': 'cmu_unitops1.CMU_controls_hxer_steptest_data_values_container.Hot Water In',
    'id': 'cmu.unitops:F1AbEcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwgPctt0f72V00TJF_JVAxrQUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVHxIT1QgV0FURVIgSU4',
    'pointsource': 'CMU_controls_hxer_steptest_d

In [20]:
for element, element_id, _ in elements_info:
    mutation = gql(
        """
mutation AssetWithDataView($from: _DatabaseInput!, $to: _ElementInput!) {
    MergeDatabaseAsset_with_dv(
        from: $from,
        to: $to
    ) {
        from {
            name
        }
        to {
            name
        }
    }
}
"""
    )
    # pprint.pprint(mutation)
    reply = client.execute(
        mutation, variable_values={"from": {"id": db_id}, "to": {"id": element_id}}
    )
    print(json.dumps(reply, indent=4))

{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name": "CMU_controls_hxer_steptest"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name": "F20_PSA_v2"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name": "F20_Wk1_PSA"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name": "F20_wk1_Membrane"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name": "HX Cycle1 F20"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "CMU_UnitOps"
        },
        "to": {
            "name

In [21]:
def dataview_streams_transaction(dv_id, dv_streams):
    if len(dv_streams) == 0:
        print(f"@@ dv_id={dv_id}")
        return 
    mutations = []
    for i, stream in enumerate(dv_streams):
        stream_mutation = """
stream{0}: 
    AddDataViewHas_stream(
        from: {{id: "{1}" }}
        to: {{id: "{2}" }}
    ) {{
        from {{
            name
        }}
        to {{
            name
        }}
    }}   
    """.format(
            i, dv_id, stream["id"]
        )
        # print("mutation", i, stream_mutation)
        mutations.append(stream_mutation)

    dv_stream_mutation = gql(
        "mutation dataview_streams {\n" + "".join(mutations) + "\n}"
    )

    reply = client.execute(dv_stream_mutation)
    print(f"[add-dv-streams: {dv_id} ({len(json.dumps(reply, indent=4))})]", end="")

In [22]:
mutation_dv_node = gql(
    """
mutation AssetDataView($asset_db: String!, $asset_id: [String]!, $columns: String!, $description: String, $id: ID!, $name: String!, $ocs_tag: String!) {
    MergeDataView(
        asset_db: $asset_db
        asset_id: $asset_id
        id: $id
        columns: $columns 
        description: $description
        name: $name
        ocs_sync: false
        ocs_tag: $ocs_tag
    ) {
        name
        id 
    }
}
    """
)


mutation_dv_rel = gql(
    """
mutation ElementDataView($from: _ElementInput!, $to: _DataViewInput!) {
    AddElementHas_dataview(
        from: $from
        to: $to
    ) {
        from {
            name
        }
        to {
            name
            id
        }
    }
}
    """
)

def dataview_id(asset_ids, ocs_tag, multiple=None):
    dv_asset = (
        multiple
        if len(asset_ids) > 1
        else asset_ids[0].lower().replace(" ", ".").replace("/", "__")
    )
    dv_id = f"{dv_id_prefix}-{dv_asset}"
    if ocs_tag != "all_columns":
        dv_id = f"{dv_id}-{ocs_tag}"
    return dv_id 

def create_asset_dataview(
    dv_name, asset_ids, elem_ids, all_streams, ocs_tag, columns, multiple=None
):
    dv_id = dataview_id(asset_ids, ocs_tag, multiple)
    asset_desc = (
        f"Asset {asset_ids[0]}" if len(asset_ids) == 1 else f"Assets {multiple.upper()}"
    )
    reply_node = client.execute(
        mutation_dv_node,
        variable_values={
            "asset_db": asset_db,
            "asset_id": asset_ids,
            "columns": str(sorted(columns))
            if len(columns) > 0
            else str(sorted(set([s["name"] for s in all_streams]))),
            "description": f"Hub DV for {asset_desc} - {dv_name}",
            "id": dv_id,
            "name": dv_name,
            "ocs_tag": ocs_tag,
        },
    )

    for elem_id in elem_ids:
        reply_rel = client.execute(
            mutation_dv_rel,
            variable_values={"from": {"id": elem_id}, "to": {"id": dv_id}},
        )
        # print(json.dumps(reply_rel, indent=4))

    if len(columns) == 0:
        dv_streams = all_streams
    else:
        dv_streams = [s for s in all_streams if s["name"] in columns]

    dataview_streams_transaction(dv_id, dv_streams)

element_ids = {}

for dv_name in dv_defs.keys():
    print(f"dv_name={dv_name}")
    ocs_tag, columns, has_custom_dv = dv_defs[dv_name]
    for asset_id, elem_id, all_streams in elements_info:
        if has_no_dv(asset_id):
            continue
        element_ids[asset_id] = elem_id
        # no specialized data view except for fermenter vessels
        if len(columns) > 0 and not has_custom_dv(asset_id):
            continue
    # print(dv_name, [asset_id], all_streams, ocs_tag, columns)
        create_asset_dataview(
            dv_name, [asset_id], [elem_id], all_streams, ocs_tag, columns
        )

dv_name=Default
[add-dv-streams: cmu.unitops-cmu_controls_hxer_steptest (1218)][add-dv-streams: cmu.unitops-f20_psa_v2 (1841)][add-dv-streams: cmu.unitops-f20_wk1_psa (143)][add-dv-streams: cmu.unitops-f20_wk1_membrane (973)][add-dv-streams: cmu.unitops-hx.cycle1.f20 (2450)][add-dv-streams: cmu.unitops-membrane (973)][add-dv-streams: cmu.unitops-wk1_f20 (2450)]

In [23]:
def extract_columns(asset_ids, ocs_tag):
    query = gql(
        """
    query DataView($id: ID!) {
        DataView(id: $id) {
            columns
      }  
    }
    """
    )
    reply = client.execute(
        query, variable_values={"id": dataview_id([asset_ids[0]], ocs_tag)}
    )
    # print(dataview_id([asset_ids[0]], ocs_tag), reply["DataView"][0]["columns"])
    return sorted(
        [
            i.strip()
            for i in reply["DataView"][0]["columns"][1:-1].replace("'", "").split(",")
        ]
    )


multi_asset_dvs = []
try:
    exec(config["multi_asset_dvs"])
except:
    pass
print(config["multi_asset_dvs"])

for asset_ids, dv_suffix in multi_asset_dvs:
    for dv_name in dv_defs.keys():
        ocs_tag, columns, has_custom_dv = dv_defs[dv_name]
        if len(columns) == 0:
            columns = extract_columns(asset_ids, ocs_tag)
        elem_ids = [element_ids[i] for i in asset_ids]
        create_asset_dataview(
            dv_name + "+", asset_ids, elem_ids, [], ocs_tag, columns, multiple=dv_suffix
        )

# No multi-asset DV



### Asset metadata gathering and update, plus geo-data if applicable

In [29]:
print("db name:", database_name)
meta_query = gql(
    """
query Database($name: String) {
  Database(name: $name) {
    asset_with_dv(orderBy: name_asc) {
      name
      id
      has_attribute(orderBy: name_asc) {
        name
        value
        type
      }
    }
  }
}
"""
)
result = client.execute(meta_query, variable_values={"name": database_name})

db name: CMU_UnitOps


In [30]:
def convert2value(s, vtype):
    if vtype == "Double":
        return float(s)
    else:
        return str(s)


def extract_meta(js, sub=False):
    j = js["has_attribute"]
    prefix = "" if not sub else (js["name"].lower() + ".")
    return {f"{prefix}{k['name']}": convert2value(k["value"], k["type"]) for k in j}

In [31]:
# test
d = extract_meta(result["Database"][0]["asset_with_dv"][0])
d

{}

In [32]:
meta_mutation = gql(
    """
mutation UpdateMeta($id: ID!, $meta: String) {
  MergeElement(id: $id, asset_metadata: $meta) {
    id
    name
    asset_metadata
  }
}
"""
)

geo_mutation = gql(
    """
mutation UpdateGeo($id: ID!, $lat: Float, $long: Float)  {
    MergeElement(id: $id, latitude: $lat, longitude: $long, location: {latitude: $lat, longitude: $long}) {
        name
        id
        latitude
        longitude
        location {
            latitude
            longitude
        }
    }
}    
"""
)


def update_meta(wid, meta):
    result = client.execute(
        meta_mutation, variable_values={"id": wid, "meta": str(meta)}
    )
    return result


def update_geo(wid, lat, long):
    result = client.execute(
        geo_mutation, variable_values={"id": wid, "lat": lat, "long": long}
    )
    return result

In [33]:
nb = len(result["Database"][0]["asset_with_dv"])
for i in range(nb):
    dai = result["Database"][0]["asset_with_dv"][i]
    d = extract_meta(dai)
    for j in range(len(dai.get("has_element", []))):
        d.update(extract_meta(dai["has_element"][j], True))
    print(dai["name"], dai["id"], d)
    update_meta(dai["id"], d)
    if config["db"].get("asset_geodata", False):
        update_geo(dai["id"], d.get("Latitude", 0.0), d.get("Longitude", 0.0))

CMU_controls_hxer_steptest cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwwoyPe5P56BGoMAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxDTVVfQ09OVFJPTFNfSFhFUl9TVEVQVEVTVA {}
F20_PSA_v2 cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwRBVNqtL56hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxGMjBfUFNBX1Yy {}
F20_Wk1_PSA cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwmtAyhkn46hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxGMjBfV0sxX1BTQQ {}
F20_wk1_Membrane cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpws6SEJRr56hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxGMjBfV0sxX01FTUJSQU5F {}
HX Cycle1 F20 cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpw3WK8ZW736hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxIWCBDWUNMRTEgRjIw {}
Membrane cmu.unitops:F1EmcFWpYyj-GkWKwoVLgdSfpwytzxkHz36hGoOAANOjL3QwUElBRi1BQ0FEXENMQVNTUk9PTSBEQVRBXFNPVVJDRSBEQVRBXENNVV9VTklUT1BTMVxNRU1CUkFORQ {}
wk1_F20 cmu.